In [24]:
using Pkg
#Pkg.add("DecisionTree")
Pkg.add("GraphViz")

Pkg.Types.PkgError: The following package names could not be resolved:
 * GraphViz (not found in project, manifest or registry)
Please specify by known `name=uuid`.

In [9]:
using CSV
using DataFrames
using PyPlot
using ScikitLearn # machine learning package
using StatsBase
using Random
using LaTeXStrings # for L"$x$" to work instead of needing to do "\$x\$"
using Printf
using DecisionTree


# (optional)change settings for all plots at once, e.g. font size
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 16

# (optional) change the style. see styles here: https://matplotlib.org/3.1.1/gallery/style_sheets/style_sheets_reference.html
PyPlot.matplotlib.style.use("seaborn-white") 

┌ Info: Precompiling DecisionTree [7806a523-6efd-50cb-b5f6-3fa6f1930dbb]
└ @ Base loading.jl:1242


The following project will be attempting to classify a species of mushroom as either poisonous or edible. The data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identied as definitely edible, definitely poisonous, or of unknown edibility and not recommended(labeled as poisonous). The goal is to use a _______ machine learning software to correctly predict whether or not a mushroom is edible. Each mushroom species in the data set has a classification along with 22 attributes listed below.

Attribute Information:

1. cap-shape: 
        b=bell
        c=conical
        x=convex
        f=flat
        k=knobbed
        s=knobbed
2. cap-surface: 
        f=fibrous
        g=grooves
        y=scaly
        s=smooth
3. cap-color: 
        n=brown
        b=buff
        c=cinnamon
        g=gray
        r=green
        p=pink
        u=purple
        e=red
        w=white
        y=yellow
4. bruises?: 
        t=bruises
        f=no
5. odor: 
        a=almond
        l=anise
        c=creosote
        y=fishy
        f=foul
        m=musty
        n=none
        p=pungent
        s=spicy
6. gill-attachment: 
        a=attached
        d=descending
        f=free
        n=notched
7. gill-spacing: 
        c=close
        w=crowded
        d=distant
8. gill-size: 
        b=broad
        n=narrow
9. gill-color: 
        k=black
        n=brown
        b=buff
        h=chocolate
        g=gray
        r=green
        o=orange
        p=pink
        u=purple
        e=red
        w=white
        y=yellow
10. stalk-shape: 
        e=enlarging
        t=tapering
11. stalk-root: 
        b=bulbous
        c=club
        u=cup
        e=equal
        z=rhizomorphs
        r=rooted
        ?=missing
12. stalk-surface-above-ring: 
        f=fibrous
        y=scaly
        k=silky
        s=smooth
13. stalk-surface-below-ring: 
        f=fibrous
        y=scaly
        k=silky
        s=smooth
14. stalk-color-above-ring: 
        n=brown
        b=buff
        c=cinnamon
        g=gray
        o=orange
        p=pink
        e=red
        w=white
        y=yellow
15. stalk-color-below-ring: 
        n=brown
        b=buff
        c=cinnamon
        g=gray
        o=orange
        p=pink
        e=red
        w=white
        y=yellow
16. veil-type: 
        p=partial
        u=universal
17. veil-color: 
        n=brown
        o=orange
        w=white
        y=yellow
18. ring-number: 
        n=none
        o=one
        t=two
19. ring-type: 
        c=cobwebby
        e=evanescent
        f=flaring
        l=large
        n=none
        p=pendant
        s=sheathing
        z=zone
20. spore-print-color: 
        k=black
        n=brown
        b=buff
        h=chocolate
        r=green
        o=orange
        u=purple
        w=white
        y=yellow
21. population: 
        a=abundant
        c=clustered
        n=numerous
        s=scattered
        v=several
        y=solitary
22. habitat: 
        g=grasses
        l=leaves
        m=meadows
        p=paths
        u=urban
        w=waste
        d=woods

The logical rules to determine whether a mushroom is edible or not that has proven to be the most successful are as follows. 

    P_1) odor=NOT(almond.OR.anise.OR.none)
	     120 poisonous cases missed, 98.52% accuracy

	P_2) spore-print-color=green
	     48 cases missed, 99.41% accuracy
         
	P_3) odor=none.AND.stalk-surface-below-ring=scaly.AND.
	          (stalk-color-above-ring=NOT.brown) 
	     8 cases missed, 99.90% accuracy
         
	P_4) habitat=leaves.AND.cap-color=white
	         100% accuracy     

	Rule P_4) may also be

	P_4') population=clustered.AND.cap_color=white

In [2]:
df_mushrooms = CSV.read("df_mushrooms.csv")

,class_p,cap_shape_x,cap_shape_b,cap_shape_s,cap_shape_f,cap_shape_k,cap_shape_c
,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,1,1,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0
4,1,1,0,0,0,0,0
5,0,1,0,0,0,0,0
6,0,1,0,0,0,0,0
7,0,0,1,0,0,0,0
8,0,0,1,0,0,0,0
9,1,1,0,0,0,0,0


I want to explore the logical rules provided to see some relationships 

In [3]:
#names of all the attributes 
attributes = names(df_mushrooms)
for i = 1:length(attributes)
    println(string(attributes[i]))
    
end
attributes


class_p
cap_shape_x
cap_shape_b
cap_shape_s
cap_shape_f
cap_shape_k
cap_shape_c
cap_surface_s
cap_surface_y
cap_surface_f
cap_surface_g
cap_color_n
cap_color_y
cap_color_w
cap_color_g
cap_color_e
cap_color_p
cap_color_b
cap_color_u
cap_color_c
cap_color_r
bruises_t
odor_p
odor_a
odor_l
odor_n
odor_f
odor_c
odor_y
odor_s
odor_m
gill_attachment_f
gill_spacing_c
gill_size_n
gill_color_k
gill_color_n
gill_color_g
gill_color_p
gill_color_w
gill_color_h
gill_color_u
gill_color_e
gill_color_b
gill_color_r
gill_color_y
gill_color_o
stalk_shape_e
stalk_root_e
stalk_root_c
stalk_root_b
stalk_root_r
stalk_root_?
stalk_surface_above_ring_s
stalk_surface_above_ring_f
stalk_surface_above_ring_k
stalk_surface_above_ring_y
stalk_surface_below_ring_s
stalk_surface_below_ring_f
stalk_surface_below_ring_y
stalk_surface_below_ring_k
stalk_color_above_ring_w
stalk_color_above_ring_g
stalk_color_above_ring_p
stalk_color_above_ring_n
stalk_color_above_ring_b
stalk_color_above_ring_e
stalk_color_above_ring_o


112-element Array{Symbol,1}:
 :cap_shape_x  
 :cap_shape_b  
 :cap_shape_s  
 :cap_shape_f  
 :cap_shape_k  
 :cap_shape_c  
 :cap_surface_s
 :cap_surface_y
 :cap_surface_f
 :cap_surface_g
 :cap_color_n  
 :cap_color_y  
 :cap_color_w  
 ⋮             
 :population_n 
 :population_a 
 :population_v 
 :population_y 
 :population_c 
 :habitat_u    
 :habitat_g    
 :habitat_m    
 :habitat_d    
 :habitat_p    
 :habitat_w    
 :habitat_l    

# Feature matrix and Target vector
* feature matrix `X`, must be `n_mushrooms` by `n_features`
* target vector `y`, must be `n_mushrooms` long 

I created these with NaN to issure I didnt skip over any data

In [6]:
#number of mushrooms species in data
n_mushrooms = nrow(df_mushrooms)

#feature matrix
X = NaN*zeros(n_mushrooms, length(attributes)-1)
#target vector
y = NaN*zeros(n_mushrooms)

#itterate through each row in the data frame
for (i, mushrooms) in enumerate(eachrow(df_mushrooms))
    
    #in each row I need to itterate over each individual attribute 
    for (j, attribute) in enumerate(attributes[2:end])
        X[i, j] = mushrooms[attribute]
    end
    
    #fill in the target vecor with whether or not the species is poisonous 
    y[i] = mushrooms[:class_p]
end


# Decision Tree 



In [12]:
#using Pruned Tree Classifier 

model = build_tree(y, X)


Decision Tree
Leaves: 12
Depth:  5

In [13]:
model_new = prune_tree(model, 0.9)


Decision Tree
Leaves: 6
Depth:  4

In [14]:
print_tree(model_new, 5)

Feature 25, Threshold 0.5
L-> Feature 48, Threshold 0.5
    L-> 1.0 : 3760/4048
    R-> 0.0 : 512/548
R-> Feature 96, Threshold 0.5
    L-> Feature 58, Threshold 0.5
        L-> 0.0 : 3392/3400
        R-> Feature 49, Threshold 0.5
            L-> 1.0 : 40/40
            R-> 0.0 : 16/16
    R-> 1.0 : 72/72


In [15]:
#apply_tree()
#use this to predict 

In [20]:
n_folds = 5
accuracy = nfoldCV_tree(y, X, n_folds, 0.9)


Fold 1
Mean Squared Error:     0.042487684729064036
Correlation Coeff:      0.9171978735849896
Coeff of Determination: 0.8295933574622101

Fold 2
Mean Squared Error:     0.04495073891625616
Correlation Coeff:      0.9118295236287878
Coeff of Determination: 0.8198995829883555

Fold 3
Mean Squared Error:     0.03140394088669951
Correlation Coeff:      0.9384237538576456
Coeff of Determination: 0.8742451963590261

Fold 4
Mean Squared Error:     0.038177339901477834
Correlation Coeff:      0.9251648740561712
Coeff of Determination: 0.8469554643562852

Fold 5
Mean Squared Error:     0.04741379310344827
Correlation Coeff:      0.9071512400329311
Coeff of Determination: 0.8103307320698623

Mean Coeff of Determination: 0.8362048666471479


5-element Array{Float64,1}:
 0.8295933574622101
 0.8198995829883555
 0.8742451963590261
 0.8469554643562852
 0.8103307320698623